In [18]:
import boto3
import pandas as pd
from io import BytesIO
from config import FeaturePartitionEnum
import generate_feature_report
from generate_feature_report import FeatureReport
from importlib import reload
s3 = boto3.client("s3")
bucket_name = "credit-risk-modeling-dj72195"

In [11]:
# Load base table
object_key = "train/base/train_base.csv"
base_feature_path = 'train/feature'

reponse = s3.get_object(Bucket=bucket_name, Key=object_key)

content = reponse['Body'].read()
base_df = pd.read_csv(BytesIO(content))


Here, we will create a table which will aid us in performing perliminary feature selection. For each feature,
the following statistics will be calculated:

- Mean
- Median
- Standard Deviation
- Weight of Evidence
- Information Value
- Correlation with Target
- Number of missing values
- Data type (categorical or numerical)
- Number of unique values (categorical)
- Min and max values (numberical)
- Skewedness
- Kurtosis

In [19]:
from importlib import reload
reload(generate_feature_report)
from generate_feature_report import FeatureReport
for feature_class in FeaturePartitionEnum:
    if feature_class.value == 'applprev':
        feature_class_partition = base_feature_path + '/' + feature_class.value
        report = FeatureReport(base_df, feature_class_partition, s3)
        report.combine_tables()

Am I getting here
How about here, if Contents in response


/Users/dustinhayes/Desktop/GitHub/stable-credit-risk-modeling/DataExploration/generate_feature_report.py:59: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(BytesIO(file_obj['Body'].read()))


   case_id  actualdpd_943P  annuity_853A approvaldate_319D  \
0        2             0.0         640.2               NaN   
1        2             0.0        1682.4               NaN   
2        3             0.0        6140.0               NaN   
3        4             0.0        2556.6               NaN   
4        5             0.0           NaN               NaN   

   byoccupationinc_3656910L cancelreason_3545846M  childnum_21L  \
0                       NaN              a55475b1           0.0   
1                       NaN              a55475b1           0.0   
2                       NaN           P94_109_143           NaN   
3                       NaN             P24_27_36           NaN   
4                       NaN           P85_114_140           NaN   

  creationdate_885D  credacc_actualbalance_314A  credacc_credlmt_575A  ...  \
0        2013-04-03                         NaN                   0.0  ...   
1        2013-04-03                         NaN                   0.

/Users/dustinhayes/Desktop/GitHub/stable-credit-risk-modeling/DataExploration/generate_feature_report.py:59: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(BytesIO(file_obj['Body'].read()))


   case_id  actualdpd_943P  annuity_853A approvaldate_319D  \
0    40704             0.0     7204.6000               NaN   
1    40734             0.0     3870.2000               NaN   
2    40737             0.0     2324.4001               NaN   
3    40791             0.0     2320.8000               NaN   
4    40791             0.0     2541.2000               NaN   

   byoccupationinc_3656910L cancelreason_3545846M  childnum_21L  \
0                       NaN           P94_109_143           NaN   
1                       NaN           P94_109_143           NaN   
2                       1.0              a55475b1           0.0   
3                       1.0              a55475b1           0.0   
4                       1.0              a55475b1           0.0   

  creationdate_885D  credacc_actualbalance_314A  credacc_credlmt_575A  ...  \
0        2018-11-20                         NaN                   0.0  ...   
1        2019-12-26                         NaN                   0.